# Reinforcement Learning with Direct Preference Optimization (DPO)

In [ ]:
# Cell Purpose: Install required packages for DPO training
# - Installs Unsloth for efficient LLM training
# - Installs TRL (Transformers Reinforcement Learning) for DPO trainer
# - Handles both local and Google Colab environments
# - Installs dependencies for 4-bit quantization and optimization

%%capture

import os
!pip install --upgrade -qqq uv

if "COLAB_" not in "".join(os.environ.keys()):
    # Local installation
    !pip install unsloth vllm
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
else:
    # Google Colab installation
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps xformers trl peft accelerate bitsandbytes

print("✅ Installation complete!")

In [ ]:
# Cell Purpose: Check GPU availability and specifications
# - Verifies CUDA GPU is available
# - Displays GPU name and total memory
# - Checks BF16 support for mixed precision training
# - Warns if GPU memory is insufficient (<6GB)

# Check GPU availability


import torch

print("🔍 GPU Information:")
print(f"  GPU Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"  GPU Name: {torch.cuda.get_device_name(0)}")
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  GPU Memory: {gpu_memory:.2f} GB")
    print(f"  BF16 Support: {torch.cuda.is_bf16_supported()}")

    if gpu_memory < 6:
        print("\n⚠️  Warning: Less than 6GB VRAM. Consider using smaller batch size or sequence length.")
else:
    print("\n⚠️  No GPU detected. DPO training will be very slow on CPU.")

🔍 GPU Information:
  GPU Available: True
  GPU Name: NVIDIA A100-SXM4-80GB
  GPU Memory: 79.32 GB
  BF16 Support: True


## Step 2: Load Preference Dataset

In [ ]:
# Cell Purpose: Load UltraFeedback preference dataset for DPO training
# - Loads high-quality preference pairs (chosen vs rejected responses)
# - Uses first 2000 samples for faster training
# - Displays example to show dataset structure
# - Shows prompt, preferred (chosen), and rejected responses

from datasets import load_dataset


print("📦 Loading UltraFeedback Binarized Preferences dataset...")
print("   This is a production-quality dataset with 60k+ samples")
print("   Loading first 2000 samples for faster training...\n")

dataset = load_dataset(
    "argilla/ultrafeedback-binarized-preferences-cleaned",
    split="train[:2000]"
)

print(f"✅ Dataset loaded successfully!")
print(f"   Total samples: {len(dataset)}")
print(f"   Features: {dataset.column_names}")

# Display a sample preference pair
print("\n" + "="*80)
print("📝 EXAMPLE PREFERENCE PAIR")
print("="*80)

sample_data = dataset[0]

# Show the prompt
print(f"\n🔵 PROMPT:")
print("-" * 80)
print(sample_data['prompt'][:500] + "..." if len(sample_data['prompt']) > 500 else sample_data['prompt'])

print(f"\n🟢 CHOSEN (Preferred Response):")
print("-" * 80)
preferred_response = sample_data['chosen'][-1]['content'] if isinstance(sample_data['chosen'], list) else sample_data['chosen']
print(preferred_response[:500] + "..." if len(preferred_response) > 500 else preferred_response)

print(f"\n🔴 REJECTED (Less Preferred Response):")
print("-" * 80)
rejected_response = sample_data['rejected'][-1]['content'] if isinstance(sample_data['rejected'], list) else sample_data['rejected']
print(rejected_response[:500] + "..." if len(rejected_response) > 500 else rejected_response)

print("\n" + "="*80)
print("💡 The model will learn to prefer 'chosen' responses over 'rejected' ones.")
print("💡 This dataset contains diverse, real-world instructions and high-quality responses.")

📦 Loading UltraFeedback Binarized Preferences dataset...
   This is a production-quality dataset with 60k+ samples
   Loading first 2000 samples for faster training...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60917 [00:00<?, ? examples/s]

✅ Dataset loaded successfully!
   Total samples: 2000
   Features: ['source', 'prompt', 'chosen', 'chosen-rating', 'chosen-model', 'rejected', 'rejected-rating', 'rejected-model']

📝 EXAMPLE PREFERENCE PAIR

🔵 PROMPT:
--------------------------------------------------------------------------------
Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here's some starter code to help you out:
#include <iostream>
#include <string>
using namespace std;
int main() {
    string country;
    // prompt user for input
    cout << "Enter the name of a country: ";
    cin >> country;
    // check if country borders the Mediterranean Sea
    // [C++ code]
    return 0;
}

🟢 CHOSEN (Preferred Response):
--------------------------------------------------------------------------------
Here's a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea:

#include <iostream>


## Step 3: Load Model with 4-bit Quantization

In [ ]:
# Cell Purpose: Load Llama-3.2-3B model with 4-bit quantization
# - Loads 3B parameter model for better DPO performance
# - Uses 4-bit quantization to fit in GPU memory
# - Configures padding token for batch processing (required for DPO)
# - DPO needs to process chosen/rejected pairs in batches

from unsloth import FastLanguageModel

# Model configuration
maximum_sequence_length = 2048  # Maximum sequence length for training
data_type = None           # Auto-detect optimal dtype (bfloat16 if supported)
quantization_4bit = True    # Enable 4-bit quantization to save memory

print("🔄 Loading model...")

# Load SmolLM2-135M with Unsloth optimizations
language_model, text_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Llama-3.2-3B",
    max_seq_length = maximum_sequence_length,
    dtype = data_type,
    load_in_4bit = quantization_4bit,
)

# Configure padding token for batch processing
# DPO requires batch processing of chosen/rejected pairs
# Padding ensures all sequences in a batch have the same length
if text_tokenizer.pad_token is None:
    text_tokenizer.pad_token = text_tokenizer.eos_token
    text_tokenizer.pad_token_id = text_tokenizer.eos_token_id
    print("✅ Padding token configured")

# Model information
total_parameters = sum(p.numel() for p in language_model.parameters())
print(f"\n✅ Model loaded: {language_model.config._name_or_path}")
print(f"   Total parameters: {total_parameters:,}")
print(f"   Max sequence length: {maximum_sequence_length}")
print(f"   4-bit quantization: {quantization_4bit}")
print(f"   Memory footprint: ~4GB VRAM")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
🔄 Loading model...
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]


✅ Model loaded: unsloth/llama-3.2-3b-unsloth-bnb-4bit
   Total parameters: 1,841,212,416
   Max sequence length: 2048
   4-bit quantization: True
   Memory footprint: ~4GB VRAM


## Step 4: Apply LoRA for Efficient DPO Training

In [ ]:
# Cell Purpose: Apply LoRA adapters optimized for DPO training
# - Uses rank=32 (higher than standard tasks) for nuanced preference learning
# - Targets attention and MLP layers
# - No dropout for DPO stability
# - Calculates parameter efficiency (~99% savings)

print("🔧 Applying LoRA adapters for DPO training...")

# Apply LoRA with configuration optimized for preference learning
language_model = FastLanguageModel.get_peft_model(
    language_model,
    r = 32,  # Higher rank for nuanced preference learning (vs 8-16 for standard tasks)
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj",     # MLP layers
    ],
    lora_alpha = 32,       # Match rank for stable DPO training
    lora_dropout = 0,       # No dropout improves DPO stability
    bias = "none",          # No bias adaptation
    use_gradient_checkpointing = "unsloth",  # Unsloth's optimized checkpointing
    random_state = 3407,    # For reproducibility
    use_rslora = False,     # Standard LoRA scaling
)

# Calculate parameter efficiency
trainable_params = sum(p.numel() for p in language_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in language_model.parameters())
trainable_percent = (trainable_params / total_parameters) * 100

print(f"\n✅ LoRA Applied Successfully!")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Total parameters: {total_parameters:,}")
print(f"   Trainable percentage: {trainable_percent:.4f}%")
print(f"   LoRA Rank: 32")
print(f"   Memory savings: ~99% fewer parameters to train!")

🔧 Applying LoRA adapters for DPO training...


Unsloth 2025.11.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.



✅ LoRA Applied Successfully!
   Trainable parameters: 48,627,712
   Total parameters: 1,889,840,128
   Trainable percentage: 2.5731%
   LoRA Rank: 32
   Memory savings: ~99% fewer parameters to train!


## Step 5: Prepare Dataset for DPO Training

In [ ]:
# Cell Purpose: Format dataset for DPO training requirements
# - Extracts prompt, chosen (preferred), and rejected responses
# - Handles both list and string formats from source data
# - Creates clean triplets required by DPOTrainer
# - Displays formatted example to verify structure

def format_for_dpo(data_example):

    # The prompt is already a clean string
    user_prompt = data_example['prompt']

    # Extract the assistant's response from chosen conversation
    # chosen/rejected are lists of message dicts with 'role' and 'content'
    if isinstance(data_example['chosen'], list):
        # Get the last assistant message
        preferred_text = [msg['content'] for msg in data_example['chosen'] if msg['role'] == 'assistant'][-1]
    else:
        preferred_text = data_example['chosen']

    if isinstance(data_example['rejected'], list):
        # Get the last assistant message
        rejected_text = [msg['content'] for msg in data_example['rejected'] if msg['role'] == 'assistant'][-1]
    else:
        rejected_text = data_example['rejected']

    return {
        'prompt': user_prompt,
        'chosen': preferred_text,
        'rejected': rejected_text,
    }

print("🔄 Formatting dataset for DPO training...")

# Apply formatting to dataset
formatted_dpo_dataset = dataset.map(
    format_for_dpo,
    remove_columns=dataset.column_names,
)

print(f"✅ Dataset formatted!")
print(f"   Samples: {len(formatted_dpo_dataset)}")
print(f"   Format: prompt + chosen + rejected")

# Show formatted example
print("\n" + "="*80)
print("📝 FORMATTED DPO EXAMPLE")
print("="*80)
formatted_example = formatted_dpo_dataset[0]
print(f"\n🔵 PROMPT:\n{formatted_example['prompt'][:400]}...\n" if len(formatted_example['prompt']) > 400 else f"\n🔵 PROMPT:\n{formatted_example['prompt']}\n")
print(f"🟢 CHOSEN:\n{formatted_example['chosen'][:400]}...\n" if len(formatted_example['chosen']) > 400 else f"🟢 CHOSEN:\n{formatted_example['chosen']}\n")
print(f"🔴 REJECTED:\n{formatted_example['rejected'][:400]}..." if len(formatted_example['rejected']) > 400 else f"🔴 REJECTED:\n{formatted_example['rejected']}")
print("="*80)

🔄 Formatting dataset for DPO training...


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Dataset formatted!
   Samples: 2000
   Format: prompt + chosen + rejected

📝 FORMATTED DPO EXAMPLE

🔵 PROMPT:
Can you write a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea? Here's some starter code to help you out:
#include <iostream>
#include <string>
using namespace std;
int main() {
    string country;
    // prompt user for input
    cout << "Enter the name of a country: ";
    cin >> country;
    // check if country borders the Mediter...

🟢 CHOSEN:
Here's a C++ program that prompts the user to enter the name of a country and checks if it borders the Mediterranean Sea:

#include <iostream>
#include <string>
#include <set>
#include <map>
#include <algorithm>

using namespace std;

int main() {
    // store countries and their bordering seas in a map
    map<string, set<string>> countries;
    countries["Algeria"] = {"Mediterranean Sea", "North...

🔴 REJECTED:
Sure, here is the program using the C++11 algorithm "cds

## Step 6: Configure and Start DPO Training

In [ ]:
# Cell Purpose: Configure DPO trainer with optimized hyperparameters
# - Sets beta=0.1 for KL divergence penalty (balances learning vs reference model)
# - Uses gradient accumulation for effective batch size of 8
# - Applies 8-bit AdamW optimizer for memory efficiency
# - Enables gradient checkpointing and mixed precision training

from trl import DPOTrainer, DPOConfig

print("⚙️  Configuring DPO Trainer...")

# DPO Training Configuration
dpo_training_config = DPOConfig(
    # Model training
    beta = 0.1,  # KL divergence penalty (higher = stay closer to reference model)

    # Optimization
    per_device_train_batch_size = 2,     # Samples per GPU
    gradient_accumulation_steps = 4,      # Effective batch size = 2 * 4 = 8
    learning_rate = 5e-5,                 # Lower LR for stable DPO training

    # Training schedule
    max_steps = 200,                      # Total training steps (increase for better results)
    warmup_steps = 10,                    # Warmup for first 10 steps

    # Logging and checkpointing
    logging_steps = 10,                   # Log every 10 steps
    save_steps = 50,                      # Save checkpoint every 50 steps
    output_dir = "./dpo_output",          # Where to save checkpoints

    # Optimization settings
    optim = "adamw_8bit",                 # 8-bit AdamW optimizer for memory efficiency
    weight_decay = 0.01,                  # L2 regularization
    lr_scheduler_type = "cosine",         # Cosine learning rate decay

    # Memory optimization
    fp16 = not torch.cuda.is_bf16_supported(),  # Use fp16 if bf16 not available
    bf16 = torch.cuda.is_bf16_supported(),       # Use bf16 if available (better precision)
    gradient_checkpointing = True,        # Trade compute for memory

    # Misc
    seed = 42,
    report_to = "none",  # Disable wandb/tensorboard for simplicity
)

# Initialize DPO Trainer
dpo_trainer = DPOTrainer(
    model = language_model,
    args = dpo_training_config,
    train_dataset = formatted_dpo_dataset,
    tokenizer = text_tokenizer,
)

print(f"✅ DPO Trainer configured!")
print(f"   Training steps: {dpo_training_config.max_steps}")
print(f"   Effective batch size: {dpo_training_config.per_device_train_batch_size * dpo_training_config.gradient_accumulation_steps}")
print(f"   Beta (KL penalty): {dpo_training_config.beta}")
print(f"   Learning rate: {dpo_training_config.learning_rate}")

⚙️  Configuring DPO Trainer...


Extracting prompt in train dataset (num_proc=16):   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=16):   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=16):   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ DPO Trainer configured!
   Training steps: 200
   Effective batch size: 8
   Beta (KL penalty): 0.1
   Learning rate: 5e-05


In [ ]:
# Cell Purpose: Execute DPO training and monitor GPU memory usage
# - Checks GPU memory before training starts
# - Trains model to prefer chosen responses over rejected ones
# - Displays progress logs every 10 steps
# - Returns training statistics upon completion

# Check memory usage before training
gpu_properties = torch.cuda.get_device_properties(0)
initial_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
maximum_memory = round(gpu_properties.total_memory / 1024 / 1024 / 1024, 3)

print(f"\n💾 Memory Status Before Training:")
print(f"   GPU: {gpu_properties.name}")
print(f"   Max memory: {maximum_memory} GB")
print(f"   Reserved: {initial_gpu_memory} GB")
print(f"   Available: {maximum_memory - initial_gpu_memory:.2f} GB")

print(f"\n🚀 Starting DPO Training...")
print(f"   This will take approximately 10-20 minutes depending on your GPU")
print(f"   Progress will be logged every 10 steps\n")

# Start training!
training_statistics = dpo_trainer.train()

print(f"\n✅ Training Complete!")

The model is already on multiple devices. Skipping the move to device specified in `args`.



💾 Memory Status Before Training:
   GPU: NVIDIA A100-SXM4-80GB
   Max memory: 79.318 GB
   Reserved: 3.252 GB
   Available: 76.07 GB

🚀 Starting DPO Training...
   This will take approximately 10-20 minutes depending on your GPU
   Progress will be logged every 10 steps



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss
10,0.684300,0.019468,0.000639,0.475000,0.018829,-419.173828,-301.204346,-0.987858,-1.042551,0,0,0
20,0.640400,0.261312,0.112740,0.725000,0.148572,-470.455322,-369.244293,-0.934998,-0.906552,No Log,No Log,No Log
30,0.551300,0.390508,-0.126452,0.712500,0.516960,-412.468079,-305.816101,-0.846624,-0.754306,No Log,No Log,No Log
40,0.732100,0.234403,-0.159046,0.612500,0.393449,-415.678894,-345.813660,-0.808216,-0.750002,No Log,No Log,No Log
50,0.566900,0.362957,-0.188336,0.750000,0.551294,-420.465393,-325.244141,-0.804373,-0.811630,No Log,No Log,No Log
60,0.613600,0.501647,0.115649,0.700000,0.385998,-408.110779,-361.932343,-0.952955,-0.870685,No Log,No Log,No Log
70,0.583400,0.517864,0.047887,0.712500,0.469977,-409.372620,-376.219208,-1.100441,-0.978914,No Log,No Log,No Log
80,0.518100,0.581043,0.014764,0.762500,0.566279,-433.305756,-354.832245,-1.131116,-1.182440,No Log,No Log,No Log
90,0.532200,0.450612,-0.099078,0.800000,0.549690,-441.855164,-331.726074,-1.228107,-1.133368,No Log,No Log,No Log
100,0.539200,0.444269,-0.230456,0.812500,0.674725,-420.487122,-328.125885,-0.926490,-0.915349,No Log,No Log,No Log



✅ Training Complete!
   Time taken: 523.31 seconds
   Time taken: 8.72 minutes


In [ ]:
# Cell Purpose: Display comprehensive training statistics and memory metrics
# - Calculates total training time and performance rates
# - Shows peak GPU memory usage and training memory overhead
# - Highlights Unsloth's speed and memory efficiency improvements
# - Provides percentage-based memory utilization metrics

# Show final memory and performance statistics
final_memory_used = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
training_memory_usage = round(final_memory_used - initial_gpu_memory, 3)
memory_usage_percentage = round(final_memory_used / maximum_memory * 100, 3)
training_memory_percentage = round(training_memory_usage / maximum_memory * 100, 3)

print(f"\n📊 Training Statistics:")
print(f"   Training runtime: {training_statistics.metrics['train_runtime']:.2f} seconds")
print(f"   Training runtime: {round(training_statistics.metrics['train_runtime']/60, 2)} minutes")
print(f"   Samples per second: {training_statistics.metrics.get('train_samples_per_second', 0):.2f}")
print(f"   Steps per second: {training_statistics.metrics.get('train_steps_per_second', 0):.2f}")

print(f"\n💾 Memory Usage:")
print(f"   Peak reserved: {final_memory_used} GB")
print(f"   Memory for training: {training_memory_usage} GB")
print(f"   Peak % of max memory: {memory_usage_percentage}%")
print(f"   Training % of max memory: {training_memory_percentage}%")

print(f"\n✨ DPO training with Unsloth:")
print(f"   ✓ 2x faster than standard implementations")
print(f"   ✓ 60% less memory usage")
print(f"   ✓ Same accuracy as full precision training")


📊 Training Statistics:
   Training runtime: 523.31 seconds
   Training runtime: 8.72 minutes
   Samples per second: 3.06
   Steps per second: 0.38

💾 Memory Usage:
   Peak reserved: 21.312 GB
   Memory for training: 18.06 GB
   Peak % of max memory: 26.869%
   Training % of max memory: 22.769%

✨ DPO training with Unsloth:
   ✓ 2x faster than standard implementations
   ✓ 60% less memory usage
   ✓ Same accuracy as full precision training


## Step 7: Test the DPO-Trained Model

In [ ]:
# Cell Purpose: Test DPO-trained model with streaming text generation
# - Enables fast inference mode for optimal generation speed
# - Uses a complex quantum computing prompt to test model quality
# - Streams generated tokens in real-time for immediate feedback
# - Demonstrates improved response quality after DPO alignment

from transformers import TextStreamer

# Enable fast inference mode
FastLanguageModel.for_inference(language_model)

print("🧪 Testing DPO-Trained Model\n")
print("="*80)

# Test prompt
first_test_prompt = """User: Explain Quantum computing and it's effect on Machine learning"""

# Tokenize the prompt
tokenized_inputs = text_tokenizer(first_test_prompt, return_tensors="pt").to("cuda")

print(f"PROMPT:\n{first_test_prompt}\n")
print("="*80)
print("MODEL RESPONSE:")
print("-"*80)

🧪 Testing DPO-Trained Model

PROMPT:
User: Explain Quantum computing and it's effect on Machine learning

MODEL RESPONSE:
--------------------------------------------------------------------------------
. [closed]
Quantum computing is a new field in computing, which is based on quantum theory. The main difference between quantum computing and classical computing is that the former uses the quantum mechanical phenomena, such as superposition and entanglement to perform computations. This makes the quantum computing more powerful than the classical computing. It has the potential to solve many problems that are intractable in classical computing, and it is also more energy-efficient.
The effect of quantum computing on machine learning is that it can improve the performance of machine learning algorithms. For example, quantum machine learning algorithms can solve problems that are intractable in classical machine learning, such as finding the optimal parameters for a machine learning algo

In [ ]:
# Cell Purpose: Test model with a second prompt for code generation
# - Uses factorial calculation prompt to evaluate coding capability
# - Generates response with temperature=0.7 for creative but focused output
# - Streams tokens in real-time using TextStreamer
# - Validates DPO improved the model's coding instruction following

# Test with another prompt
print("\n\n" + "="*80)
second_test_prompt = """User: Write a short Python function to calculate factorial."""

tokenized_inputs_2 = text_tokenizer(second_test_prompt, return_tensors="pt").to("cuda")

print(f"PROMPT:\n{second_test_prompt}\n")
print("="*80)
print("MODEL RESPONSE:")
print("-"*80)

response_streamer_2 = TextStreamer(text_tokenizer, skip_prompt=True)
generated_outputs_2 = language_model.generate(
    **tokenized_inputs_2,
    streamer=response_streamer_2,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
)

print("\n" + "="*80)



PROMPT:
User: Write a short Python function to calculate factorial.

MODEL RESPONSE:
--------------------------------------------------------------------------------
 The function should take a single integer as input and return the factorial of that integer.
Input: A single integer n
Output: The factorial of n
Example: factorial(5) returns 120 (5! = 1*2*3*4*5)
Hint: Factorial can be calculated recursively by the formula:
n! = n * (n-1)!
Or, in code:
def factorial(n):
    if n==1:
    return n * factorial(n-1)
Hint: Factorial can also be calculated iteratively, by the formula:
n! = n * (n-1) * (n-2) *... * 1
Or, in code:
def factorial(n):
    result = 1
    for i in range(1, n+1):
        result *= i
    return result
Hint: Factorial can be calculated by using the formula:
n! = n * (n-1)!
Or, in code



## Step 8: Save the Fine-tuned Model

Let's save our DPO-trained model so we can use it later!

In [ ]:
# Cell Purpose: Save DPO-trained model with LoRA adapters for future use
# - Saves LoRA adapter weights (smaller than full model)
# - Saves tokenizer configuration and vocabulary
# - Provides option to merge adapters with base model for standalone deployment
# - Enables easy sharing and reloading of the trained model

# Save the model locally
output_model_path = "./smollm2_dpo_model"

print(f"💾 Saving DPO-trained model to {output_model_path}...")

# Save LoRA adapters
language_model.save_pretrained(output_model_path)
text_tokenizer.save_pretrained(output_model_path)

print(f"✅ Model saved successfully!")
print(f"   Location: {output_model_path}")
print(f"   Files saved: adapter_config.json, adapter_model.safetensors, tokenizer files")

# Optional: Merge LoRA adapters with base model for easier deployment
print(f"\n🔀 You can also merge LoRA adapters with base model:")
print(f"   language_model.save_pretrained_merged('{output_model_path}_merged', text_tokenizer)")
print(f"   This creates a single model file without adapters.")

💾 Saving DPO-trained model to ./smollm2_dpo_model...
✅ Model saved successfully!
   Location: ./smollm2_dpo_model
   Files saved: adapter_config.json, adapter_model.safetensors, tokenizer files

🔀 You can also merge LoRA adapters with base model:
   model.save_pretrained_merged('./smollm2_dpo_model_merged', tokenizer)
   This creates a single model file without adapters.
